In [2]:
from utils import load_data
import datasets
import torch, spacy
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# this part is the same as in "calculating_embeddings" to load the datasets

# get device proper device to run on
if torch.cuda.is_available():
    device = "cuda"
else:
    # calculating the embeddings on a cpu can take hours, not recommended
    device = "cpu"
all_records_df = load_data("../data/")

# find and delete null values
print("counting null values")
print(all_records_df.isnull().sum())

all_records_df.dropna(axis=0, inplace=True)

# also we want a df of individual videos that are identified by title and description
videos_df = all_records_df.drop_duplicates(["title", "description"], keep="last")
# note: if you want the first record in videos_df, enter vidoes_df.iloc[0] not videos_df[0]
# as the later will try to return the record that has index or id = 0  wich is the index in the all_records_df

counting null values
title                             1
views                             0
time-stamp-upload-milliseconds    0
time-stamp                        0
date-time-hr                      0
upload-time-hr                    0
upload-time-stamp                 0
description                       0
video-length                      0
video-length-milliseconds         0
channel                           0
dtype: int64


In [4]:
print(videos_df)

                                                    title   views   
28      Twenty years on John Bolton is still defending...   20438  \
29      Paul Bremer on Iraq occupation mistakes and ac...    2829   
88      Indian opposition leader Rahul Gandhi disquali...   29224   
89      India’s Rahul Gandhi disqualified from parliament   64930   
179     TikTok CEO Shou Zi Chew faces off with US legi...    7052   
...                                                   ...     ...   
228295  How does Bill Clinton's tapes-in-sock-drawer c...  218547   
228296  Judge Jeanine: Biden knows he has a lot to hid...  146345   
228297  What about the 17 recordings implicating Presi...   41617   
228298  Jesse Watters: Biden banned a trans flasher fr...  438730   
228299    JD Vance: The DOJ going after Trump is ‘insane’   37934   

        time-stamp-upload-milliseconds     time-stamp   
28                            87102000  1679754890966  \
29                            87119000  1679754890967   


In [9]:
videos_df.sample(n=round(videos_df.shape[0]*0.1), random_state = 100).to_csv(r'..\data\supervised_training.csv', index=False)
# Label the topics manually and save it in another csv file...

In [12]:
# videos_df.sample(n=200, random_state = 200).to_csv(r'..\data\supervised_training_gpt.csv', index=False)
# chat GPT geht nicht

In [7]:
# get the labelled data for training
df = pd.read_csv('..\data\supervised_training_labelled.csv')

# tokenize using SpaCy
nlp = spacy.load("en_core_web_lg")
df['text'] = df['title'] + " " + df['description']
df['tokens'] = [nlp(text).vector for text in df['text']]

# convert tokens to a 2D array
train_vectors = df['tokens'].to_numpy()

# prepare target labels
labels = df['topic'].to_numpy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_vectors, labels, test_size=0.2, random_state=42)